# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics, GLPK
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))
;

In [2]:
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [3]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [[33.0 2 "Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 [33.0 2 "Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 [33.0 2 "Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 [33.0 2 "Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 [33.0 2 "Feeder_450" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

# services = [[33.0 2 "Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
#             [33.0 2 "Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]

services = [
    [200000 1 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [130000 1 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [97000 1 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [100000 1 "Feeder_800" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]


# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [4]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [5]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
    end
end

In [6]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end

edges = vcat(ports_adj, s_t_adj, transshipment_adj, penalty_adj, dummy_adj);

In [7]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

In [8]:
function dijkstra(G, source, destination)
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

## Flowproblem

### Initialization

In [9]:
current_paths = []
current_costs = []
current_commodities = []
;

## Basis

In [10]:
# Bases med penalty edges
for i in 1:length(penalty_adj)
    push!(current_paths, penalty_adj[i][1:2])
    push!(current_costs, penalty_adj[i][3])
    append!(current_commodities, i)
end


# Edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (ports_adj)
        push!(i_j, string(edge[1], "|", edge[2]))
    end
end

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of penalty paths

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [11]:
# Initial base including penalty edges
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path
mvs = Matrix{Float64}(undef, n_mv, 0) # Type of vessel on each commodity
aij = Matrix{Float64}(undef, n_ij, 0)
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

# Constraint matrices for commodidty paths PK and edges used
for i in P
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(current_paths[i])
        if string(current_paths[i][j-1], "|", current_paths[i][j]) ∈ i_j
            ak[findall(x -> x == string(current_paths[i][j-1], "|", current_paths[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
end

# Constraint matrix for edge capacities
for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

# Constraint matrix for service vessels
for i in services
    mv = zeros(Float64, n_mv)
    mv[findall(x -> x == i[3], baltic_fleet."Vessel class")[1]] = 1
    mvs = [mvs mv]
end

P = 1:length(current_paths)
A = 1:size(aij, 2)

1:22

### Basis problem with penalty only

In [12]:
flowModel = Model(GLPK.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
#print(flowModel)
optimize!(flowModel)
solution_summary(flowModel)

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K] # Dual variables for commodities at iteration 1 
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij] # Dual variables for edges at iteration 1 
reduced_cost = [-1.0] # Kickstarting while loop
;


In [17]:
#while any(reduced_cost .< 0)

append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))

# Initializing feasible paths
candidate_paths = []
candidate_costs = []
candidate_commodities = []

# Cost for shortest path
cij = []

# Setting up new network with reduced cost
edges_w_dual = deepcopy(edges)
for i in 1:length(edges_w_dual)
    edges_w_dual[i][3] = edges_w_dual[i][3] - dual_Y_ij[i]
end

for i in 1:size(baltic_demand, 1)
    c = 0
    path, cost = dijkstra(get_G(edges_w_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    for j in 2:length(path)
        for edge in edges
            if edge[1] == path[j-1] && edge[2] == path[j]
                c += edge[3]
            end
        end
    end
    push!(candidate_costs, c)
    #lav et for loop der tjekker omkostningen fra i,j i path
    push!(candidate_paths, path)
    push!(cij, cost)
    append!(candidate_commodities, i)
end

# bevar candidate cost da vi bruger den her, men costen i vores model/objektfunktion skal være de originale omkostninger fra 'edges'
reduced_cost = cij - dual_Y_k - baltic_demand[:,4]


cp_r_k = candidate_paths[reduced_cost .< 0] #candidate_paths[reduced_cost .== minimum(reduced_cost)]

for i in 1:length(cp_r_k)
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(cp_r_k[i])
        if string(cp_r_k[i][j-1], "|", cp_r_k[i][j]) ∈ i_j
            ak[findall(x -> x == string(cp_r_k[i][j-1], "|", cp_r_k[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
    push!(current_paths, cp_r_k[i])
    push!(current_costs, candidate_costs[reduced_cost .< 0][i])
    append!(current_commodities, findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1])
end

# Cost på ij skal være fra originalt netværk (vi skal ikke have reducd cost med)

P = 1:length(current_paths)
n_p = length(current_paths)
A = 1:size(aij, 2)

revenue = zeros(Int64, n_ff)
append!(revenue, [baltic_demand[current_commodities[i],4][1] for i in n_ff+1:n_p])


flowModel = Model(GLPK.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum((current_costs[j]-revenue[j])*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
optimize!(flowModel)
# print(flowModel)

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]
println(reduced_cost)

println(dual_Y_ij)

# end
# primal_x = value.(x[P])
# primal_y = value.(y[S])


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-341.44444444444434, 0.0, 0.0, -103.0, -554.0, 0.0, 0.0, 0.0, -591.0, -358.0, 0.0, 0.0, -357.99999999999994, -358.0, -159.0, -198.99999999999997]


In [19]:
print(flowModel)

Min 200000 y[1] + 130000 y[2] + 97000 y[3] + 100000 y[4] + 1000 x[1] + 1000 x[2] + 1000 x[3] + 1000 x[4] + 1000 x[5] + 1000 x[6] + 1000 x[7] + 1000 x[8] + 1000 x[9] + 1000 x[10] + 1000 x[11] + 1000 x[12] + 1000 x[13] + 1000 x[14] + 1000 x[15] + 1000 x[16] + 1000 x[17] + 1000 x[18] + 1000 x[19] + 1000 x[20] + 1000 x[21] + 1000 x[22] - 711 x[23] - 148 x[24] - 522 x[25] - 804 x[26] - 713 x[27] - 743 x[28] - 287 x[29] - 1452 x[30] - 424 x[31] - 721 x[32] - 526 x[33] - 572 x[34] - 780 x[35] - 183 x[36] - 663 x[37] - 307 x[38] - 62 x[39] - 800 x[40] - 518 x[41] - 107 x[42] - 506 x[43] - 277 x[44] - 120 x[45] + 210 x[46] - 250 x[47] - 40 x[48] - 287 x[49] + 130 x[50] - 130 x[51] - 150 x[52] - 130 x[53] - 80 x[54] + 40 x[55] + 220 x[56] - 150 x[57] + 410 x[58] + 140 x[59] + 240 x[60] - 610 x[61] + 240 x[62] + 120 x[63] - 160 x[64] - 363 x[65] - 307 x[66] + 36 x[67]
Subject to
 

Y_k[1] : x[1] + x[23] + x[45] = 77
 Y_k[2] : x[2] + x[24] + x[46] = 456
 Y_k[3] : x[3] + x[25] = 65
 Y_k[4] : x[4] + x[26] + x[47] = 7
 Y_k[5] : x[5] + x[27] + x[61] = 10
 Y_k[6] : x[6] + x[28] + x[48] = 98
 Y_k[7] : x[7] + x[29] + x[49] + x[62] = 660
 Y_k[8] : x[8] + x[30] = 17
 Y_k[9] : x[9] + x[31] + x[50] = 268
 Y_k[10] : x[10] + x[32] + x[51] = 18
 Y_k[11] : x[11] + x[33] + x[63] = 16
 Y_k[12] : x[12] + x[34] + x[52] = 37
 Y_k[13] : x[13] + x[35] + x[53] = 187
 Y_k[14] : x[14] + x[36] + x[54] = 50
 Y_k[15] : x[15] + x[37] + x[55] = 231
 Y_k[16] : x[16] + x[38] + x[56] + x[66] = 597
 Y_k[17] : x[17] + x[39] = 32
 Y_k[18] : x[18] + x[40] + x[57] = 162
 Y_k[19] : x[19] + x[41] + x[64] = 397
 Y_k[20] : x[20] + x[42] + x[58] + x[67] = 1215
 Y_k[21] : x[21] + x[43] + x[59] + x[65] = 6
 Y_k[22] : x[22] + x[44] + x[60] = 298
 Y_ij[1] : x[25] + x[27] + x[30] - 450 y[1] ≤ 0
 Y_ij[2] : x[27] + x[30] + x[39] - 450 y[1] ≤ 0
 Y_ij[3] : x[27] + x[34] + x[39] - 450 y[1] ≤ 0
 Y_ij[4] : x[34] + x[3

 x[14] ≥ 0
 x[15] ≥ 0
 x[16] ≥ 0
 x[17] ≥ 0
 x[18] ≥ 0
 x[19] ≥ 0
 x[20] ≥ 0
 x[21] ≥ 0
 x[22] ≥ 0
 x[23] ≥ 0
 x[24] ≥ 0
 x[25] ≥ 0
 x[26] ≥ 0
 x[27] ≥ 0
 x[28] ≥ 0
 x[29] ≥ 0
 x[30] ≥ 0
 x[31] ≥ 0
 x[32] ≥ 0
 x[33] ≥ 0
 x[34] ≥ 0
 x[35] ≥ 0
 x[36] ≥ 0
 x[37] ≥ 0
 x[38] ≥ 0
 x[39] ≥ 0
 x[40] ≥ 0
 x[41] ≥ 0
 x[42] ≥ 0
 x[43] ≥ 0
 x[44] ≥ 0
 x[45] ≥ 0
 x[46] ≥ 0
 x[47] ≥ 0
 x[48] ≥ 0
 x[49] ≥ 0
 x[50] ≥ 0
 x[51] ≥ 0
 x[52] ≥ 0
 x[53] ≥ 0
 x[54] ≥ 0
 x[55] ≥ 0


 x[56] ≥ 0
 x[57] ≥ 0
 x[58] ≥ 0
 x[59] ≥ 0
 x[60] ≥ 0
 x[61] ≥ 0
 x[62] ≥ 0
 x[63] ≥ 0
 x[64] ≥ 0
 x[65] ≥ 0
 x[66] ≥ 0
 x[67] ≥ 0
 y[1] ≥ 0
 y[2] ≥ 0
 y[3] ≥ 0
 y[4] ≥ 0
 y[1] ≤ 1
 y[2] ≤ 1
 y[3] ≤ 1
 y[4] ≤ 1


In [41]:
objective_value(flowModel)

-649559.5555555555

In [26]:
print([current_paths current_commodities ])


Any[Any["FIRAUs", "DEBRVt"] 1; Any["DEBRVs", "DKAARt"] 2; Any["DEBRVs", "NOSVGt"] 3; Any["RUKGDs", "DEBRVt"] 4; Any["DEBRVs", "NOAESt"] 5; Any["DEBRVs", "PLGDYt"] 6; Any["SEGOTs", "DEBRVt"] 7; Any["DEBRVs", "NOBGOt"] 8; Any["DEBRVs", "RUKGDt"] 9; Any["DEBRVs", "FIRAUt"] 10; Any["NOKRSs", "DEBRVt"] 11; Any["NOBGOs", "DEBRVt"] 12; Any["DEBRVs", "FIKTKt"] 13; Any["NOAESs", "DEBRVt"] 14; Any["PLGDYs", "DEBRVt"] 15; Any["DEBRVs", "SEGOTt"] 16; Any["NOSVGs", "DEBRVt"] 17; Any["FIKTKs", "DEBRVt"] 18; Any["DKAARs", "DEBRVt"] 19; Any["DEBRVs", "RULEDt"] 20; Any["DEBRVs", "NOKRSt"] 21; Any["RULEDs", "DEBRVt"] 22; Any["FIRAUs", "FIRAU", "PLGDY", "DEBRV2", "DEBRVt"] 1; Any["DEBRVs", "DEBRV3", "DKAAR", "DKAARt"] 2; Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"] 3; Any["RUKGDs", "RUKGD", "FIKTK", "FIRAU", "PLGDY", "DEBRV2", "DEBRVt"] 4; Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOAES", "NOAESt"] 5; Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "PLGDY", "PLGDYt"] 6; Any["SEGOTs", "SEGOT4", "RULED",

In [93]:
objective_value(flowModel)

229640.88888888888

In [130]:
[current_paths current_commodities ] |> DisplayAs.Unlimited

UndefVarError: UndefVarError: `DisplayAs` not defined

In [90]:
println(current_paths[x1])
println(current_paths[y1])

Any[

"DEBRVs", "RULEDt"]
Any["DEBRVs", "DEBRV4", "SEGOT4", "RULED", "RULEDt"]


In [91]:
println(value.(x[x1]) + value.(x[y1]))

1215.0


In [92]:
println(value.(x[y1]))

203.0


In [30]:
baltic_demand

Row,Origin,Destination,FFEPerWeek,Revenue_1,TransitTime
,String7,String7,Int64,Int64,Int64
1,FIRAU,DEBRV,77,1120,16
2,DEBRV,DKAAR,456,790,13
3,DEBRV,NOSVG,65,1050,14
4,RUKGD,DEBRV,7,1250,31
5,DEBRV,NOAES,10,1610,20
6,DEBRV,PLGDY,98,1040,10
7,SEGOT,DEBRV,660,760,10
8,DEBRV,NOBGO,17,2030,30
9,DEBRV,RUKGD,268,870,12


In [48]:
test = zeros(Int64, n_k)

append!(test, current_commodities[n_ff+1:n_p])

44-element Vector{Any}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [213]:
candidate_paths[reduced_cost .< 0]

7-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [222]:
candidate_paths[reduced_cost .< 0]

7-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [131]:
solution_summary(flowModel)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 3.42021e+06
  Objective bound    : -Inf
  Dual objective value : 3.42021e+06

* Work counters
  Solve time (sec)   : 1.89781e-04


In [132]:
value.(x[P])


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
And data, a 47-element Vector{Float64}:
  20.0
  12.0
   0.0
   7.0
  10.0
   0.0
   0.0
   0.0
 121.0
   0.0
   ⋮
  32.0
 162.0
 276.0
 203.0
   6.0
 298.0
 158.0
   0.0
   0.0

In [133]:
value.(y[S])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
 0.18222222222222223
 1.0
 1.0
 1.0

In [42]:
candidate_paths[reduced_cost .< 0]

8-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["SEGOTs", "SEGOT3", "NOKRS", "DEBRV3", "DEBRVt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [15]:
dual_Y_ij[dual_Y_ij .< 0]

8-element Vector{Float64}:
 -341.4444444444446
 -102.99999999999994
 -553.9999999999998
 -591.0000000000002
 -358.0
 -358.0
 -517.0
 -358.0

In [71]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
#@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
#@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
#@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
#optimize!(flowModel)

# dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
# dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]
# append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))


MethodError: MethodError: no method matching zero(::Type{Any})

Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period
   @ Dates /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Dates/src/periods.jl:51
  zero(!Matched::GenericAffExpr)
   @ JuMP ~/.julia/packages/JuMP/ToPd2/src/aff_expr.jl:201
  ...


In [30]:
current_commodities[111:end]

14-element Vector{Any}:
 22
  1
  4
  6
  7
  9
 10
 11
 13
 15
 16
 18
 20
 22

In [35]:
i_j

62-element Vector{Any}:
 "DEBRVs|DEBRV1"
 "DEBRV1|DEBRVt"
 "DEBRVs|DEBRV2"
 "DEBRV2|DEBRVt"
 "DEBRVs|DEBRV3"
 "DEBRV3|DEBRVt"
 "DEBRVs|DEBRV4"
 "DEBRV4|DEBRVt"
 "NOSVGs|NOSVG"
 "NOSVG|NOSVGt"
 ⋮
 "FIRAU|PLGDY"
 "PLGDY|DEBRV2"
 "DEBRV3|DKAAR"
 "DKAAR|SEGOT3"
 "SEGOT3|NOKRS"
 "NOKRS|DEBRV3"
 "DEBRV4|SEGOT4"
 "SEGOT4|RULED"
 "RULED|DEBRV4"

Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.

In [ ]:
# edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (penalty_adj ∪ dummy_adj)
        break
    end
    push!(i_j, string(edge[1], "|", edge[2]))
end

# Paths and costs from shortest path
for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(current_paths, path)
    push!(current_costs, cost)
    append!(current_commodity, i)
end

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of ff paths - Could be hard coded

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p
index_p = n_ff+1:n_p  # Index of non-ff paths - Could be hardcoded

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [ ]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, n_ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

for i in index_p
    ak = zeros(Float64, n_ij)
    for j in 2:length(paths[i])
        ak[findall(x -> x == string(paths[i][j-1], "|", paths[i][j]), i_j)[1]] = 1
    end
    aij = [aij ak]
end


for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end



In [ ]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(costs[j]*x[j] for j in P))
#@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in K) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
print(flowModel)
optimize!(flowModel)
#solution_summary(flowModel)
;